In [ ]:
'''
Python for Data Analysis
Data QCing exercises

Graeme Hawker, University of Strathclyde
2018-07-23
'''

import numpy as np
import pandas as pd

In [ ]:
#Create a DataFrame from the data in the ‘LDT SCADA Data Example BAD’ csv file.
turbine_data_bad = pd.read_csv('LDT SCADA Data Example BAD.csv', index_col=0, parse_dates=True)
turbine_data_bad

In [ ]:
#Some missing rows are contained in the ‘LDT SCADA Data Example MISSING’ csv file. 
turbine_data_missing = pd.read_csv('LDT SCADA Data Example MISSING.csv', index_col=0, parse_dates=True)
turbine_data_missing

In [ ]:
#Create a single DataFrame containing the combined data from the two files, 
#including checking for and removing any duplication of rows.
turbine_data_combined = pd.concat([turbine_data_bad,turbine_data_missing])
turbine_data_combined

In [ ]:
turbine_data_combined_deduplicated = turbine_data_combined[~turbine_data_combined.index.duplicated()]
turbine_data_combined_deduplicated

In [ ]:
#Some rows are still blank and contain no values for any column other than the index. 
#Create a new DataFrame with these rows removed entirely.
turbine_data_combined_deduplicated_dropped = turbine_data_combined_deduplicated.dropna(how='all')
turbine_data_combined_deduplicated_dropped

In [ ]:
#Check the Power_kW column for spurious values. 
#Create a new column with these values corrected appropriately.
MAX_POWER_KW = 2000
MIN_POWER_KW = -10
turbine_data_combined_deduplicated_dropped[~turbine_data_combined_deduplicated_dropped['Power_kW'].between(MIN_POWER_KW,MAX_POWER_KW)]

In [ ]:
turbine_data_corrected = turbine_data_combined_deduplicated_dropped.copy()
turbine_data_corrected['Bad_Power_kW'] = ~turbine_data_combined_deduplicated_dropped['Power_kW'].between(MIN_POWER_KW,MAX_POWER_KW)
turbine_data_corrected

In [ ]:
turbine_data_corrected['New_Power_kW'] = turbine_data_corrected['Power_kW'][turbine_data_corrected.Bad_Power_kW==False]
turbine_data_corrected[turbine_data_corrected['Bad_Power_kW']==True]

In [ ]:
turbine_data_corrected['New_Power_kW']=turbine_data_corrected['New_Power_kW'].interpolate()
turbine_data_corrected[turbine_data_corrected['Bad_Power_kW']==True]

In [ ]:
#Save the original and corrected data as two sheets in a single Excel workbook. 
writer = pd.ExcelWriter('turbine_data_example.xlsx')
turbine_data_combined.to_excel(writer,'Raw Data')
turbine_data_corrected.to_excel(writer,'Corrected Data')
writer.save()